In [468]:
import keras, numpy as np, matplotlib.pyplot as plt
from keras.datasets import mnist

(x_train,y_train),(x_test,y_test) = mnist.load_data()

x_train = x_train.astype('float')/255
x_test = x_test.astype('float')/255

x_train = np.reshape(x_train,[60000,784])
y_train = np.reshape(y_train,[60000,1])

x_test = np.reshape(x_test,[10000,784])
y_test = np.reshape(y_test,[10000,1])

z = np.zeros((60000,10))
for i in range(60000):
    z[i][y_train[i]] = 1
y_train = z

z = np.zeros((10000,10))
for i in range(10000):
    z[i][y_test[i]] = 1
y_test = z

del z

In [488]:
weights1 = np.random.rand(784,128) * 0.01
weights2 = np.random.rand(128,64) * 0.01
weights3 = np.random.rand(64,10) * 0.01
bias1 = np.zeros((1,128))
bias2 = np.zeros((1,64))
bias3 = np.zeros((1,10))

In [489]:
def normalize(x):
    num = (x - np.min(x,axis=1).reshape(len(x),1))
    den = (np.max(x,axis=1).reshape(len(x),1) - np.min(x,axis=1).reshape(len(x),1))
    return num/den

In [490]:
def softmax(x):
    X = normalize(x)
    e_x = np.exp(X) 
    soft = e_x / (np.sum(e_x,axis=1).reshape(len(x),1))
    return soft

In [491]:
def tanh(x):
    X = normalize(x)
    t = (np.exp(X)-np.exp(-X)) / (np.exp(X)+np.exp(-X))
    return t

In [492]:
def dtanh(x):
    t=tanh(x)
    dt=1-t**2
    return dt

In [493]:
def update(x,y,w1,b1,w2,b2,w3,b3,lr):
    
    z1 = np.dot(x,w1) + b1
    a1 = np.array(tanh(z1))
    
    z2 = np.dot(a1,w2) + b2
    a2 = np.array(tanh(z2))
    
    z3 = np.dot(a2,w3) + b3
    a3 = np.array(softmax(z3))
    
    dz3 = a3-y
    dw3 = (1/len(x)) * np.dot(a2.T,dz3)
    db3 = (1/len(x)) * np.sum(dz3,axis=0,keepdims=True)
    
    dz2 = np.dot(dz3,w3.T) * dtanh(z2)
    dw2 = (1/len(x)) * np.dot(a1.T,dz2)
    db2 = (1/len(x)) * np.sum(dz2,axis=0,keepdims=True)
    
    dz1 = np.dot(dz2,w2.T) * dtanh(z1)
    dw1 = (1/len(x)) * np.dot(x.T,dz1)
    db1 = (1/len(x)) * np.sum(dz1,axis=0,keepdims=True)
    
    w1 -= lr*dw1
    b1 -= lr*db1
    w2 -= lr*dw2
    b2 -= lr*db2
    w3 -= lr*dw3
    b3 -= lr*db3
    
    return [w1,b1,w2,b2,w3,b3]

In [494]:
for j in range(60):
    x_new = np.array([x_train[i] for i in range((1000*j),(1000*(j+1)))]).reshape(1000,784)
    y_new = np.array([y_train[i] for i in range((1000*j),(1000*(j+1)))]).reshape(1000,10)
    [weights1,bias1,weights2,bias2,weights3,bias3] = update(x_new,y_new,weights1,bias1,weights2,bias2,weights3,bias3,0.001)

In [495]:
print(bias1)

[[-2.42101574e-07 -2.25697204e-07  8.18725125e-08 -3.41576102e-07
  -1.69577233e-07 -2.14653696e-07 -1.56452018e-07 -1.91557940e-07
  -3.39647437e-07 -2.99908430e-07  1.70241058e-07 -3.07094278e-07
  -3.79842824e-07 -1.21960637e-07 -3.14534654e-07 -9.70768208e-08
  -3.44158284e-07 -3.11060564e-07  1.00634319e-07 -3.64142495e-07
  -2.75307459e-07 -1.85913171e-07  2.72570947e-08 -1.52803702e-07
  -3.79299115e-07 -7.41921183e-08 -5.46429248e-07 -2.64510720e-07
  -4.42988083e-07 -2.44068764e-07 -3.40050117e-07  1.85064815e-08
  -6.52886182e-08 -1.64560332e-07 -4.21815794e-07 -3.89461064e-07
  -5.11159218e-07 -3.33839282e-07 -6.12562580e-08 -3.91025748e-07
  -1.31877950e-07 -1.53727593e-07 -2.02347316e-07 -2.15957061e-07
  -1.93701683e-07 -2.39403204e-07 -2.88375390e-07 -1.13465122e-08
  -2.47777838e-07 -1.96786122e-07 -4.51735032e-08  5.21314378e-08
  -1.35485382e-07 -1.93844720e-07 -3.45114643e-07  2.67892732e-08
  -3.71755906e-08 -3.52400534e-07 -2.02511725e-07 -2.48741177e-07
  -2.34636

In [496]:
xt_new = np.array([x_test[i] for i in range(1000)]).reshape(1000,784)
yt_new = np.array([y_test[i] for i in range(1000)]).reshape(1000,10)

zt1 = np.dot(x_new,weights1) + bias1
at1 = np.array(tanh(zt1))

zt2 = np.dot(at1,weights2) + bias2
at2 = np.array(tanh(zt2))

zt3 = np.dot(at2,weights3) + bias3
at3 = np.array(softmax(zt3))

predictions = np.argmax(at3,axis=1).reshape(len(at3),1)

truth = np.argmax(y_test,axis=1).reshape(len(y_test),1)

print(predictions)

[[7]
 [4]
 [3]
 [7]
 [2]
 [3]
 [1]
 [5]
 [0]
 [0]
 [8]
 [2]
 [6]
 [8]
 [0]
 [6]
 [3]
 [9]
 [5]
 [7]
 [1]
 [7]
 [8]
 [9]
 [1]
 [1]
 [2]
 [1]
 [2]
 [0]
 [8]
 [1]
 [0]
 [2]
 [0]
 [0]
 [2]
 [9]
 [4]
 [0]
 [7]
 [2]
 [3]
 [6]
 [2]
 [4]
 [3]
 [2]
 [7]
 [6]
 [1]
 [9]
 [2]
 [2]
 [7]
 [7]
 [7]
 [4]
 [1]
 [5]
 [2]
 [5]
 [3]
 [7]
 [7]
 [6]
 [9]
 [1]
 [9]
 [1]
 [3]
 [4]
 [2]
 [6]
 [2]
 [7]
 [0]
 [7]
 [5]
 [1]
 [7]
 [7]
 [1]
 [3]
 [1]
 [2]
 [5]
 [1]
 [7]
 [7]
 [7]
 [1]
 [3]
 [0]
 [7]
 [9]
 [8]
 [4]
 [6]
 [9]
 [3]
 [7]
 [2]
 [5]
 [3]
 [9]
 [1]
 [7]
 [1]
 [1]
 [5]
 [1]
 [4]
 [5]
 [8]
 [7]
 [2]
 [5]
 [3]
 [3]
 [3]
 [5]
 [4]
 [7]
 [7]
 [6]
 [2]
 [1]
 [7]
 [4]
 [4]
 [1]
 [5]
 [8]
 [0]
 [1]
 [8]
 [7]
 [7]
 [1]
 [1]
 [5]
 [4]
 [5]
 [1]
 [6]
 [6]
 [1]
 [9]
 [2]
 [2]
 [4]
 [7]
 [2]
 [6]
 [9]
 [2]
 [2]
 [9]
 [7]
 [6]
 [7]
 [1]
 [7]
 [1]
 [9]
 [8]
 [5]
 [3]
 [7]
 [3]
 [0]
 [7]
 [4]
 [1]
 [9]
 [2]
 [2]
 [3]
 [7]
 [6]
 [4]
 [1]
 [8]
 [1]
 [7]
 [5]
 [5]
 [2]
 [8]
 [3]
 [1]
 [0]
 [3]
 [1]
 [3]
 [2]
 [3]
 [7]
 [2]
